In [1]:
# standard imports
import pickle
import os
import scipy
from scipy import stats
import numpy as np
import pandas as pd
import xgboost as xgb
import sklearn
from sklearn.model_selection import train_test_split
import xarray as xr
import datetime
import matplotlib.dates as mdates

from numpy import errstate,isneginf,array
import yaml

import seaborn as sns
import cmocean as cm            # really nice colorbars
import matplotlib.pyplot as plt # for making plots

In [2]:
####need to figure out this coding for where things are stored:

In [3]:
#This file contains configuration details like API keys and passwords
global_vars = yaml.safe_load(open('../config.yml', 'r') )

#This has custom functions - log transform
# %run '/home/ds4114/repos/LDEO_Ocean_CO2_Residual/code/00_custom_functions.ipynb'
%run './00_custom_functions.ipynb'
#more functions for flux conversions
%run './00_co2_flux_equations.ipynb'

In [4]:
#Set base folders
result_folder = global_vars['reconstruction_folder']
data_folder_root = global_vars['download_folder']
print(result_folder)

#This variable sets the output file type. 
#When using cloud storage, it is recommended to use ARCO (Analysis-Ready Cloud-Optimized) formats like Zarr over NetCDF
output_file_type = '.zarr' if data_folder_root[0:5] == 'gs://' else '.nc'
print(f'Files will be outputed as: {output_file_type}')

/data/artemis/workspace/afay/LDEO_HPD/data/
Files will be outputed as: .nc


In [7]:
# Dates of Reconstruction 
# date_range_start = '1959-01-01T00:00:00.000000000'
date_range_start = '1982-01-01T00:00:00.000000000'
date_range_end = '2023-12-01T00:00:00.000000000'

# create date vector
dates = pd.date_range(start=date_range_start, 
                      end=date_range_end,freq='MS') + np.timedelta64(14, 'D')

# Dates of Reconstruction for back intime
# date_range_start = '1959-01-01T00:00:00.000000000'
date_range_start_back_time_time = '1959-01-01T00:00:00.000000000'

# create date vector
dates_back_in_time = pd.date_range(start=date_range_start_back_time_time, 
                      end=date_range_end,freq='MS') + np.timedelta64(14, 'D')

yr_end = 2023

### Calculate reconstructed fCO2 from models +error

In [8]:
####CHANGE TO MATCH NEW GOBM FILE EACH YEAR

#2024 models
models = ['cnrm', 'fesom2', 'ipsl', 'princeton', 
               'mri','noresm', 'planktom',
             'cesm_ethz','mpi','access']

#2023 models
# models = [ 'CNRM-ESM2-1', #v2023 models
#                'FESOM2-REcoM',
#                'IPSL',
#                'MOM6-Princeton', 
#                'MRI-ESM2-2',
#                'NorESM-OC1.2',
#                'CESM-ETHZ',
#                'ACCESS']

model_count = 10 #just hardcode here the number of models that go into this version

In [9]:

ds = xr.load_dataset(f"{result_folder}/HPD_LEAP_fco2-error-reconstructed-10-GOBMs_2024models_198201-202312.nc")


In [11]:
# Initialize array to include all model reconstructions...
# Since some values are NaN in some grid locations, we cannot just sum the pCO2s and divide by number of models
recon_array = np.empty(shape=(model_count,dates.size,180,360))

In [12]:
m=0
for mod in models:
    
    print(mod)
    # tmp = xr.load_dataset(f"{recon_output_dir}/{mod}_recon_198201-201912.nc")
    
    # Want pCO2 reconstruction, which is error + model
    # df['error'] =  df['pCO2'] - df[f"{mod}"]
    guess = ds[f"{mod}"] + ds[f"{mod}_error_correction"]
    # guess = (tmp[f"{mod}"] + tmp[f"error_{mod}"]).transpose("time","ylat","xlon")
    # del tmp
    
    recon_array[m,:,:,:] = guess
    
    del guess
    m +=1 

cnrm
fesom2
ipsl
princeton
mri
noresm
planktom
cesm_ethz
mpi
access


In [16]:
recon_pCO2 = np.nanmean(recon_array,axis=0) #takes the mean over the models
#annoying, but this ends up being a numpy array

<ipython-input-16-9d97a3eb2b40>:1: RuntimeWarning: Mean of empty slice
  recon_pCO2 = np.nanmean(recon_array,axis=0) #takes the mean over the models


### Back in time calculation

In [19]:
#climatology of reconstruction, by model
clim_corr_2000s = np.empty(shape=(model_count,12,180,360)) #this is what galen wants us to use now!

In [20]:
# Get climatology of corrections #
##################################
m=0
for mod in models:
    
    print(mod)
    clim_corr_2000s[m,:,:,:]= ds[f"{mod}_error_correction"].transpose("time","ylat","xlon").sel(time=slice('2000-01-01',dates[-1])).groupby("time.month").mean("time")
    # del tmp
        
    m +=1 

cnrm
fesom2
ipsl
princeton
mri
noresm
planktom
cesm_ethz
mpi
access


In [36]:
ds.dims

Frozen(SortedKeysDict({'ylat': 180, 'time': 504, 'xlon': 360}))

In [22]:
month = [1,2,3,4,5,6,7,8,9,10,11,12]
print(month)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


#### reconstruct fCO2 back in time

In [28]:
#use clim correction for years before 1982
pCO2_cc_2000s = np.empty(shape=(model_count,dates_back_in_time.size,180,360))

#for comparison, we will create a version that calculates fCO2 reconstruction if we used the constant climatology correciton for ALL years
pCO2_cc_2000s_all = np.empty(shape=(model_count,dates_back_in_time.size,180,360))

In [29]:
#need to load in full models that go back to 1959 for use here:

models_full= ds = xr.load_dataset(f"{data_folder_root}/GOBM/processed/GOBM_GCB-2024_fco2-10-models_195901-202312.nc")


In [31]:
##################################
#dm is the model array above.
m=0
for mod in models:
    
    print(mod)
    
    for yr in range(1959,1982):
        #so this is doing, for 1959-1982, adjusting the model with the climatology of the correction created by teh HPD approach
        pCO2_cc_2000s[m,(yr-1959)*12:(yr-1958)*12,:,:] = clim_corr_2000s[m,:,:,:] + models_full[f"{mod}"].sel(time=slice(f"{yr}-01-01",f"{yr}-12-31"))
        
    for yr in range(1959,yr_end):
        #this is adjusting the model with the climatology of the correction created by the HPD approach for the entire time series.
        pCO2_cc_2000s_all[m,(yr-1959)*12:(yr-1958)*12,:,:] = clim_corr_2000s[m,:,:,:] + models_full[f"{mod}"].sel(time=slice(f"{yr}-01-01",f"{yr}-12-31"))
        
    m +=1 

cnrm
fesom2
ipsl
princeton
mri
noresm
planktom
cesm_ethz
mpi
access


In [32]:
#### Fill in 1982-end with LDEO-HPD results (this is saved in recon_array above for the indiv models and recon_pCO2 for the mean):
print(recon_array.shape) #ensure this is 480 months long and put in at end of time series here for the models.
l = len(recon_array[1]) #find the length of the time dimension here to use in this loop

m = 0
for mod in models:
    print(mod)
    pCO2_cc_2000s[m,-l::,:,:] = recon_array[m,:,:,:]
    m+=1

(10, 504, 180, 360)
cnrm
fesom2
ipsl
princeton
mri
noresm
planktom
cesm_ethz
mpi
access


In [33]:
#take mean over models for reconstruction back in time as well
recon_pCO2_back_in_time = np.nanmean(pCO2_cc_2000s,axis=0) #takes the mean over the models

<ipython-input-33-d1bf17e22ac3>:2: RuntimeWarning: Mean of empty slice
  recon_pCO2_back_in_time = np.nanmean(pCO2_cc_2000s,axis=0) #takes the mean over the models
<ipython-input-33-d1bf17e22ac3>:3: RuntimeWarning: Mean of empty slice
  recon_pCO2_back_in_time_1990_2009 = np.nanmean(pCO2_cc_1990,axis=0) #takes the mean over the models


## Save netcdf of HPD back in time

In [35]:
# Create DataSets and Write out NETCDFs for Reconstructed pCO2:
final_result_folder = global_vars['hpd_save_folder']
# recon_output_dirAF = "/data/artemis/workspace/afay/LDEO_HPD/models/XGB/GCB_2022"


ds3_out = xr.Dataset({
                        'HPD_pCO2_back_in_time':(["time","lat","lon"],recon_pCO2_back_in_time.data),
                        'pCO2':(["model","time","lat","lon"],pCO2_cc_2000s.data),
                        'pCO2cc':(["model","time","lat","lon"],pCO2_cc_2000s_all.data),
                        'correction':(["model","month","lat","lon"],clim_corr_2000s)},
                        coords={'model':(['model'],models),
                        'month':(['month'],range(1,13)),
                        'time': (['time'],dates_back_in_time),
                        'lat': (['lat'],ds.ylat.data),
                        'lon':(['lon'],ds.xlon.data)})
        
# Save to netcdf
ds3_out['HPD_pCO2_back_in_time'].attrs['description'] = "Mean of model corr recons, clim of corr (2000-end) used for pCO2 prior to 1982"
ds3_out['HPD_pCO2_back_in_time'].attrs['units'] = "uatm"
ds3_out['pCO2'].attrs['description'] = "Final recon for each model, clim of corr (2000-end) used for pCO2 prior to 1982"
ds3_out['pCO2'].attrs['units'] = "uatm"
ds3_out['pCO2cc'].attrs['description'] = "Final recon for each model, Clim of corr (2000-end) used for pCO2 ENTIRE TIME"
ds3_out['pCO2cc'].attrs['units'] = "uatm"
ds3_out['correction'].attrs['description'] = "The climatology of correction (2000-end) that is used for pCO2 recon (pCO2 = correction + model)"
ds3_out['correction'].attrs['units'] = "uatm"
ds3_out.attrs['title']="LDEO-HPD Clim Correct 2000-end"
ds3_out.attrs['history']="XGBoost results and Clim Corrections by AFay"
ds3_out.attrs['institution']="Lamont Doherty Earth Observatory at Columbia"

# ds3_out.attrs['references']="/home/afay/LDEO_HPD/Back_in_Time/recon_pCO2_CO2flux_v2022.ipynb"
ds3_out.attrs['references']="/home/afay/LDEO_HPD/devans_code/05_Produce_product_back_in_time.ipynb"
ds3_out.attrs['date_created']=str(datetime.datetime.now())
ds3_out.to_netcdf(f'{final_result_folder}/GCB_2024/HPD_pCO2_cc2000s_1x1_recon_1959-2023.nc') 
